
# ref

Kipf and Welling (2016): <https://arxiv.org/abs/1609.02907>

In [1]:
import torch
import torch_geometric

# layer-wise progagation rule

Kipf and Welling (2016) use following layer-wise propagation rule:

$$H^{(l+1)} = \sigma\big(\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}H^{(l)}W^{(l)}\big).$$

Check followings:

-   In Kipf and Welling (2016) research, they suppose ${\cal G}$ as
    undirected graph.
-   $\tilde{A}=A+I_{N}$.
-   $\tilde{D}_{ii}=\sum_{j}\tilde{A}_{ij}$.
-   $W^{(l)}$ is a trainable weight matrix in $l$-th layer.
-   $H^{(l)}$ is output of $l$-th layer; $H^{(0)}=X$.

## 예제

`-` data

In [2]:
x = torch.tensor([[20],
                  [21],
                  [19],
                  [1],
                  [2],
                  [1]], dtype=torch.float)

edge_index = torch.tensor([[0, 1, 2, 0, 1, 2, 3, 4, 3, 5, 4, 5],
                           [1, 0, 0, 2, 2, 1, 4, 3, 5, 3, 5, 4]], dtype=torch.long)

y = torch.tensor([1,1,1,0,0,0], dtype=torch.int64)
data = torch_geometric.data.Data(x=x, edge_index=edge_index, y=y) 
#data.train_mask = torch.tensor([True,False,True,True,False,True])
#data.test_mask = torch.tensor([False,True,False,False,True,False])
data

Data(x=[6, 1], edge_index=[2, 12], y=[6])

> <span style="color:blue"> x를 fraud데이터로 생각하면 거래량으로 보고 y를 사기여부로 보자.

`-` GCNConv

> <span style="color:blue"> 논문에서 (8)식의 Z를 의미..

In [3]:
gconv = torch_geometric.nn.GCNConv(1,4)
gconv

GCNConv(1, 4)

In [4]:
gconv(data.x, data.edge_index)

tensor([[12.2357, -7.3451, -6.1192, -4.1723],
        [12.2357, -7.3451, -6.1192, -4.1723],
        [12.2357, -7.3451, -6.1192, -4.1723],
        [ 0.8157, -0.4897, -0.4079, -0.2782],
        [ 0.8157, -0.4897, -0.4079, -0.2782],
        [ 0.8157, -0.4897, -0.4079, -0.2782]], grad_fn=<AddBackward0>)

> <span style="color:blue"> 6X4의 피처, 위 값은 엣지가 비슷해서

In [6]:
list(gconv.parameters())

[Parameter containing:
 tensor([0., 0., 0., 0.], requires_grad=True),
 Parameter containing:
 tensor([[ 0.6118],
         [-0.3673],
         [-0.3060],
         [-0.2086]], requires_grad=True)]

> <span style="color:blue"> Bias와 weight

In [7]:
_,W = list(gconv.parameters())
W

Parameter containing:
tensor([[ 0.6118],
        [-0.3673],
        [-0.3060],
        [-0.2086]], requires_grad=True)

In [8]:
A = torch.tensor([[0., 1., 1., 0., 0., 0.],
                  [1., 0., 1., 0., 0., 0.],
                  [1., 1., 0., 0., 0., 0.],
                  [0., 0., 0., 0., 1., 1.],
                  [0., 0., 0., 1., 0., 1.],
                  [0., 0., 0., 1., 1., 0.]])
Atilde = A+torch.eye(6)
Atilde

tensor([[1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.]])

In [9]:
Atilde@data.x@W.T/3, gconv(data.x,data.edge_index)

(tensor([[12.2357, -7.3451, -6.1192, -4.1723],
         [12.2357, -7.3451, -6.1192, -4.1723],
         [12.2357, -7.3451, -6.1192, -4.1723],
         [ 0.8157, -0.4897, -0.4079, -0.2782],
         [ 0.8157, -0.4897, -0.4079, -0.2782],
         [ 0.8157, -0.4897, -0.4079, -0.2782]], grad_fn=<DivBackward0>),
 tensor([[12.2357, -7.3451, -6.1192, -4.1723],
         [12.2357, -7.3451, -6.1192, -4.1723],
         [12.2357, -7.3451, -6.1192, -4.1723],
         [ 0.8157, -0.4897, -0.4079, -0.2782],
         [ 0.8157, -0.4897, -0.4079, -0.2782],
         [ 0.8157, -0.4897, -0.4079, -0.2782]], grad_fn=<AddBackward0>))

> <span style="color:blue"> torch에서는 w를 transfo
    
> <span style="color:blue"> 3으로 나눈 부분이 $\tilde D^{-1/2} A \tilde D^{-1/2} = \tilde A /3$

`-` 즉 아래의 수식에서

$$H^{(l+1)} = \sigma\big(\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}H^{(l)}W^{(l)}\big).$$

$\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}H^{(l)}W^{(l)}$를 계산하는
Layer가 `torch_geometric.nn.GCNConv()` 으로 구현되어있음.

# Spectral graph convolutions (Section 2.1-2)

`-` In this chapter, Kipf and Welling (2016) argues why the calculation

$$H^{(l+1)} = \sigma\big(\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}H^{(l)}W^{(l)}\big).$$

can be considered as graph convolution.

`-` The properties of convolution operator $\star$ in classical spectral
analysis are as follows (3):

$$g_{\theta} \star x = Ug_{\theta}U^\top x$$

> <span style="color:blue"> $g_{\theta} \star x = Conv(x)$라고 생각하자.
    
> <span style="color:blue"> 위 식은 원래 있떤..    

where $g_{\theta}=\text{diag}(\theta)$ and $U$ is eigenvector matrix of
$L=I-D^{-1/2}AD^{-1/2}$.

> <span style="color:blue"> L을 고유분해 하면 나오는 고유벡터 U
    
`-` Now let’s examine equation (7):

$$g_{\theta}\star x \approx \theta \big(I+D^{-1/2}AD^{-1/2}\big)x. \cdots (7)$$

By expressing equation (7) matrix form, we get equation (8):

$$Z = \tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}X \Theta. \cdots (8)$$

This can be transfromed to

$$H^{(l+1)} = \sigma\big(Z\big)= \sigma\big(\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}H^{(l)}W^{(l)}\big).$$

where we interpret $H^{(l)}$ as $X$ and $W^{(l)}$ as $\Theta$.

Kipf, Thomas N, and Max Welling. 2016. “Semi-Supervised Classification
with Graph Convolutional Networks.” *arXiv Preprint arXiv:1609.02907*.